In [2]:
import duckdb as db
import pandas as pd

In [38]:
db_trips = pd.read_parquet('green_tripdata_2025-11.parquet')
db_zones = pd.read_csv('taxi_zone_lookup.csv')

In [5]:
db_trips.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [ ]:
# question 3: For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', 
# exclusive of the upper bound), how many trips had a trip_distance of less than or equal to 1 mile?
db.sql("""
       SELECT 
       COUNT(*) AS trip_count                
       FROM db_trips 
       WHERE DATE(lpep_pickup_datetime) BETWEEN '2025-11-01' AND '2025-11-30'
       AND trip_distance <= 1
       ;
       """
)

┌────────────┐
│ trip_count │
│   int64    │
├────────────┤
│       8007 │
└────────────┘

In [ ]:
# question 4: Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles (to exclude data errors).
db.sql("""
       SELECT 
       date(lpep_pickup_datetime) AS pickup_date,
       max(trip_distance) AS max_distance                
       FROM db_trips 
       WHERE trip_distance < 100
       group by pickup_date
       order by max_distance DESC
       limit 1
       ;
       """
)

┌─────────────┬──────────────┐
│ pickup_date │ max_distance │
│    date     │    double    │
├─────────────┼──────────────┤
│ 2025-11-14  │        88.03 │
└─────────────┴──────────────┘

In [ ]:
# question 5: Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?
db.sql("""
       SELECT
       zone,
       sum(total_amount) as total_amount
       FROM db_trips as trips 
       JOIN db_zones as zones on trips.PULocationID = zones.LocationID
       WHERE DATE(lpep_pickup_datetime) = '2025-11-18'
       group by zone
       order by total_amount DESC
       limit 1;
       ;
       """
)

┌───────────────────┬───────────────────┐
│       Zone        │   total_amount    │
│      varchar      │      double       │
├───────────────────┼───────────────────┤
│ East Harlem North │ 9281.919999999998 │
└───────────────────┴───────────────────┘

In [57]:
# question 6: Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?
db.sql("""
       WITH east_harlem_trips AS (
       SELECT
       DOLocationID,
       max(tip_amount) as max_tip_amount
       FROM db_trips as trips 
       JOIN db_zones as zones on trips.PULocationID = zones.LocationID
       WHERE zone = 'East Harlem North'
       group by DOLocationID, tip_amount
       order by max_tip_amount DESC
       limit 1
       )
       SELECT
       zone,
       max_tip_amount
       FROM east_harlem_trips
       LEFT JOIN db_zones as zones on east_harlem_trips.DOLocationID = zones.LocationID  
       ;
       """
)

┌────────────────┬────────────────┐
│      Zone      │ max_tip_amount │
│    varchar     │     double     │
├────────────────┼────────────────┤
│ Yorkville West │          81.89 │
└────────────────┴────────────────┘